<a href="https://colab.research.google.com/github/yakaralar/IE492/blob/master/bitirme_notebook_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import glob
import shutil
os.listdir()

['.config', 'drive', 'sample_data']

In [0]:
shutil.copy("drive/My Drive/colab_datas/IE 492/weather_turkey_processed.zip","./")
!unzip weather_turkey_processed.zip

In [0]:
shutil.copy("drive/My Drive/colab_datas/IE 492/UECM-15012015-20022020.csv","./")
shutil.copy("drive/My Drive/colab_datas/IE 492/RealTimeConsumption-01122014-09032020.csv","./")
shutil.copy("drive/My Drive/colab_datas/IE 492/all_data.feather","./")

'./all_data.feather'

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
%matplotlib inline
from tqdm import tqdm_notebook as tqdm

In [0]:
weather_paths=glob.glob("weather_turkey_processed/*.csv")
np.random.choice(weather_paths,10)

array(['weather_turkey_processed/36_35.csv',
       'weather_turkey_processed/41.5_36.5.csv',
       'weather_turkey_processed/41_40.5.csv',
       'weather_turkey_processed/40.5_45.csv',
       'weather_turkey_processed/38_28.5.csv',
       'weather_turkey_processed/38_42.5.csv',
       'weather_turkey_processed/40.5_34.csv',
       'weather_turkey_processed/41_29.5.csv',
       'weather_turkey_processed/37_27.csv',
       'weather_turkey_processed/41_33.csv'], dtype='<U38')

In [0]:
# weather_turkey=pd.DataFrame()
# for path in tqdm(weather_paths):
#     df=pd.read_csv(path)
#     weather_turkey=pd.concat([weather_turkey,df]).reset_index(drop=True)
# weather_turkey

In [0]:
# weather_turkey.to_feather("drive/My Drive/colab_datas/IE 492/all_data.feather") 

# Energy consumption

In [0]:
consumption_df=pd.read_csv("RealTimeConsumption-01122014-09032020.csv",encoding='latin-1')
consumption_df.columns=["Date","Hour","Consumption"]
consumption_df["Datetime"]=consumption_df["Date"]+" "+consumption_df["Hour"]
consumption_df["Consumption"]=consumption_df["Consumption"].apply(lambda x:x.replace(",","")).astype(float)
consumption_df["Datetime"]=pd.to_datetime(consumption_df["Datetime"])
consumption_df

,Date,Hour,Consumption,Datetime
0,31.12.2015,00:00,29590.88,2015-12-31 00:00:00
1,31.12.2015,01:00,27785.43,2015-12-31 01:00:00
2,31.12.2015,02:00,26516.53,2015-12-31 02:00:00
3,31.12.2015,03:00,26091.62,2015-12-31 03:00:00
4,31.12.2015,04:00,25871.54,2015-12-31 04:00:00
...,...,...,...,...
36715,09.03.2020,19:00,0.00,2020-09-03 19:00:00
36716,09.03.2020,20:00,0.00,2020-09-03 20:00:00
36717,09.03.2020,21:00,0.00,2020-09-03 21:00:00
36718,09.03.2020,22:00,0.00,2020-09-03 22:00:00


In [0]:
## bruuuuuuuh
#consumption_df=pd.read_csv("UECM-15012015-20022020.csv",encoding='latin-1')
#consumption_df.columns=["Date","Hour","Consumption"]
#consumption_df["Datetime"]=consumption_df["Date"]+" "+consumption_df["Hour"]
#consumption_df["Datetime"]=pd.to_datetime(consumption_df["Datetime"])
#consumption_df.Consumption=consumption_df.Consumption.apply(lambda x: x.replace(".","").replace(",",".")).astype(float)
#consumption_df

In [0]:
consumption_df=consumption_df.loc[consumption_df["Datetime"]<pd.to_datetime("2020-03-04 00:00:00")]
consumption_df

,Date,Hour,Consumption,Datetime
0,31.12.2015,00:00,29590.88,2015-12-31 00:00:00
1,31.12.2015,01:00,27785.43,2015-12-31 01:00:00
2,31.12.2015,02:00,26516.53,2015-12-31 02:00:00
3,31.12.2015,03:00,26091.62,2015-12-31 03:00:00
4,31.12.2015,04:00,25871.54,2015-12-31 04:00:00
...,...,...,...,...
36571,03.03.2020,19:00,37936.79,2020-03-03 19:00:00
36572,03.03.2020,20:00,37553.37,2020-03-03 20:00:00
36573,03.03.2020,21:00,36581.73,2020-03-03 21:00:00
36574,03.03.2020,22:00,35434.36,2020-03-03 22:00:00


# Evaluation function

In [0]:
from sklearn.metrics import mean_squared_error,mean_squared_log_error,mean_absolute_error
eval_df=[]
def eval_function(name_,y_true, y_pred):
    print("RMSE: ",np.sqrt(mean_squared_error(y_true,y_pred)))
    print("RMSLE: ",np.sqrt(mean_squared_log_error(y_true,y_pred)))
    print("MAE: ",mean_absolute_error(y_true,y_pred))
    return name_, np.sqrt(mean_squared_error(y_true,y_pred)),np.sqrt(mean_squared_log_error(y_true,y_pred)),mean_absolute_error(y_true,y_pred)

# Naive Approach 2 Days

In [0]:
cons_lagged=consumption_df.copy()

In [0]:
cons_lagged["Consumption_2_lag"]=cons_lagged["Consumption"].shift(24*2)
cons_lagged=cons_lagged.dropna()
cons_lagged

,Date,Hour,Consumption,Datetime,Consumption_2_lag
48,02.01.2016,00:00,26224.60,2016-02-01 00:00:00,29590.88
49,02.01.2016,01:00,24708.58,2016-02-01 01:00:00,27785.43
50,02.01.2016,02:00,23771.58,2016-02-01 02:00:00,26516.53
51,02.01.2016,03:00,22921.29,2016-02-01 03:00:00,26091.62
52,02.01.2016,04:00,22870.89,2016-02-01 04:00:00,25871.54
...,...,...,...,...,...
36571,03.03.2020,19:00,37936.79,2020-03-03 19:00:00,35049.21
36572,03.03.2020,20:00,37553.37,2020-03-03 20:00:00,34901.84
36573,03.03.2020,21:00,36581.73,2020-03-03 21:00:00,34225.14
36574,03.03.2020,22:00,35434.36,2020-03-03 22:00:00,33257.06


In [0]:
eval_df.append(eval_function("naive_2",cons_lagged["Consumption"],cons_lagged["Consumption_2_lag"]))

RMSE:  3606.6493919355044
RMSLE:  0.11286520561416101
MAE:  2511.0528404809397


# Naive Approach 1 Week

In [0]:
cons_lagged=consumption_df.copy()

In [0]:
cons_lagged["Consumption_7_lag"]=cons_lagged["Consumption"].shift(24*7)
cons_lagged=cons_lagged.dropna()
cons_lagged

,Date,Hour,Consumption,Datetime,Consumption_7_lag
168,07.01.2016,00:00,28763.95,2016-07-01 00:00:00,29590.88
169,07.01.2016,01:00,27284.84,2016-07-01 01:00:00,27785.43
170,07.01.2016,02:00,26321.95,2016-07-01 02:00:00,26516.53
171,07.01.2016,03:00,25748.49,2016-07-01 03:00:00,26091.62
172,07.01.2016,04:00,25636.58,2016-07-01 04:00:00,25871.54
...,...,...,...,...,...
36571,03.03.2020,19:00,37936.79,2020-03-03 19:00:00,39283.70
36572,03.03.2020,20:00,37553.37,2020-03-03 20:00:00,38791.40
36573,03.03.2020,21:00,36581.73,2020-03-03 21:00:00,37588.86
36574,03.03.2020,22:00,35434.36,2020-03-03 22:00:00,36290.64


In [0]:
eval_df.append(eval_function("naive_7",cons_lagged["Consumption"],cons_lagged["Consumption_7_lag"]))

RMSE:  2771.8255600552093
RMSLE:  0.0896548159911968
MAE:  1567.3023635201246


# Linear Regression

## Regression with 2 days lag

In [0]:
cons_lagged=consumption_df.copy()

In [0]:
cons_lagged["Consumption_2_lag"]=cons_lagged["Consumption"].shift(24*2)
cons_lagged=cons_lagged.dropna()
cons_lagged

,Date,Hour,Consumption,Datetime,Consumption_2_lag
48,02.01.2016,00:00,26224.60,2016-02-01 00:00:00,29590.88
49,02.01.2016,01:00,24708.58,2016-02-01 01:00:00,27785.43
50,02.01.2016,02:00,23771.58,2016-02-01 02:00:00,26516.53
51,02.01.2016,03:00,22921.29,2016-02-01 03:00:00,26091.62
52,02.01.2016,04:00,22870.89,2016-02-01 04:00:00,25871.54
...,...,...,...,...,...
36571,03.03.2020,19:00,37936.79,2020-03-03 19:00:00,35049.21
36572,03.03.2020,20:00,37553.37,2020-03-03 20:00:00,34901.84
36573,03.03.2020,21:00,36581.73,2020-03-03 21:00:00,34225.14
36574,03.03.2020,22:00,35434.36,2020-03-03 22:00:00,33257.06


In [0]:
train_df=cons_lagged.loc[cons_lagged["Datetime"]<=pd.to_datetime("2019-03-03 23:00:00")]
valid_df=cons_lagged.loc[cons_lagged["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]

In [0]:
from sklearn.linear_model import LinearRegression

cons_lagged["Consumption_2_lag"]=cons_lagged["Consumption"].shift(24*2)
cons_lagged=cons_lagged.dropna()

model=LinearRegression()
model.fit(train_df["Consumption_2_lag"].values.reshape(-1, 1),train_df["Consumption"].values.reshape(-1, 1))
preds=model.predict(valid_df["Consumption_2_lag"].values.reshape(-1, 1))

In [0]:
eval_df.append(eval_function("LR_2",valid_df["Consumption"],preds))

RMSE:  3424.5456173909047
RMSLE:  0.10564275989304554
MAE:  2524.446218082396


## Regression with 7 days lag

In [0]:
cons_lagged=consumption_df.copy()

In [0]:
cons_lagged["Consumption_7_lag"]=cons_lagged["Consumption"].shift(24*7)
cons_lagged=cons_lagged.dropna()
cons_lagged

,Date,Hour,Consumption,Datetime,Consumption_7_lag
168,07.01.2016,00:00,28763.95,2016-07-01 00:00:00,29590.88
169,07.01.2016,01:00,27284.84,2016-07-01 01:00:00,27785.43
170,07.01.2016,02:00,26321.95,2016-07-01 02:00:00,26516.53
171,07.01.2016,03:00,25748.49,2016-07-01 03:00:00,26091.62
172,07.01.2016,04:00,25636.58,2016-07-01 04:00:00,25871.54
...,...,...,...,...,...
36571,03.03.2020,19:00,37936.79,2020-03-03 19:00:00,39283.70
36572,03.03.2020,20:00,37553.37,2020-03-03 20:00:00,38791.40
36573,03.03.2020,21:00,36581.73,2020-03-03 21:00:00,37588.86
36574,03.03.2020,22:00,35434.36,2020-03-03 22:00:00,36290.64


In [0]:
train_df=cons_lagged.loc[cons_lagged["Datetime"]<=pd.to_datetime("2019-03-03 23:00:00")]
valid_df=cons_lagged.loc[cons_lagged["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]

In [0]:
from sklearn.linear_model import LinearRegression

cons_lagged["Consumption_7_lag"]=cons_lagged["Consumption"].shift(24*2)
cons_lagged=cons_lagged.dropna()

model=LinearRegression()
model.fit(train_df["Consumption_7_lag"].values.reshape(-1, 1),train_df["Consumption"].values.reshape(-1, 1))
preds=model.predict(valid_df["Consumption_7_lag"].values.reshape(-1, 1))

In [0]:
eval_df.append(eval_function("LR_7",valid_df["Consumption"],preds))

RMSE:  2880.6934692731347
RMSLE:  0.09102130950449729
MAE:  1834.349205329185


## Regression with both 2 and 7 days

In [0]:
cons_lagged=consumption_df.copy()

In [0]:
cons_lagged["Consumption_2_lag"]=cons_lagged["Consumption"].shift(24*2)
cons_lagged["Consumption_7_lag"]=cons_lagged["Consumption"].shift(24*7)
cons_lagged=cons_lagged.dropna()
cons_lagged

,Date,Hour,Consumption,Datetime,Consumption_2_lag,Consumption_7_lag
168,07.01.2016,00:00,28763.95,2016-07-01 00:00:00,29244.38,29590.88
169,07.01.2016,01:00,27284.84,2016-07-01 01:00:00,27637.23,27785.43
170,07.01.2016,02:00,26321.95,2016-07-01 02:00:00,26335.78,26516.53
171,07.01.2016,03:00,25748.49,2016-07-01 03:00:00,25870.23,26091.62
172,07.01.2016,04:00,25636.58,2016-07-01 04:00:00,25975.07,25871.54
...,...,...,...,...,...,...
36571,03.03.2020,19:00,37936.79,2020-03-03 19:00:00,35049.21,39283.70
36572,03.03.2020,20:00,37553.37,2020-03-03 20:00:00,34901.84,38791.40
36573,03.03.2020,21:00,36581.73,2020-03-03 21:00:00,34225.14,37588.86
36574,03.03.2020,22:00,35434.36,2020-03-03 22:00:00,33257.06,36290.64


In [0]:
train_df=cons_lagged.loc[cons_lagged["Datetime"]<=pd.to_datetime("2019-03-03 23:00:00")]
valid_df=cons_lagged.loc[cons_lagged["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]

In [0]:
from sklearn.linear_model import LinearRegression
cons_lagged=cons_lagged.dropna()

model=LinearRegression()
model.fit(train_df[["Consumption_7_lag","Consumption_2_lag"]].values,train_df["Consumption"].values.reshape(-1, 1))
preds=model.predict(valid_df[["Consumption_7_lag","Consumption_2_lag"]].values)

In [0]:
eval_df.append(eval_function("LR_2_7",valid_df["Consumption"],preds))

RMSE:  2592.2263305746233
RMSLE:  0.08182116225045082
MAE:  1700.4782113762747


## LR with dates

In [0]:
cons_lagged=consumption_df.copy()
cons_lagged["Consumption_2_lag"]=cons_lagged["Consumption"].shift(24*2)
cons_lagged["Consumption_7_lag"]=cons_lagged["Consumption"].shift(24*7)
cons_lagged["Hour"]=cons_lagged["Hour"].apply(lambda x:x.split(":")[0]).astype(int)
cons_lagged["Day"]=cons_lagged["Date"].apply(lambda x:x.split(".")[0]).astype(int)
cons_lagged["Month"]=cons_lagged["Date"].apply(lambda x:x.split(".")[1]).astype(int)
cons_lagged["weekday"]=pd.to_datetime(cons_lagged["Date"]).apply(lambda x: x.weekday()).astype(int)
cons_lagged=cons_lagged.dropna()
cons_lagged

,Date,Hour,Consumption,Datetime,Consumption_2_lag,Consumption_7_lag,Day,Month,weekday
168,07.01.2016,0,28763.95,2016-07-01 00:00:00,29244.38,29590.88,7,1,4
169,07.01.2016,1,27284.84,2016-07-01 01:00:00,27637.23,27785.43,7,1,4
170,07.01.2016,2,26321.95,2016-07-01 02:00:00,26335.78,26516.53,7,1,4
171,07.01.2016,3,25748.49,2016-07-01 03:00:00,25870.23,26091.62,7,1,4
172,07.01.2016,4,25636.58,2016-07-01 04:00:00,25975.07,25871.54,7,1,4
...,...,...,...,...,...,...,...,...,...
36571,03.03.2020,19,37936.79,2020-03-03 19:00:00,35049.21,39283.70,3,3,1
36572,03.03.2020,20,37553.37,2020-03-03 20:00:00,34901.84,38791.40,3,3,1
36573,03.03.2020,21,36581.73,2020-03-03 21:00:00,34225.14,37588.86,3,3,1
36574,03.03.2020,22,35434.36,2020-03-03 22:00:00,33257.06,36290.64,3,3,1


In [0]:
train_df=cons_lagged.loc[cons_lagged["Datetime"]<=pd.to_datetime("2019-03-03 23:00:00")]
valid_df=cons_lagged.loc[cons_lagged["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]

In [0]:
from sklearn.linear_model import LinearRegression

cons_lagged=cons_lagged.dropna()

cols=train_df.columns.difference(["Date","Consumption","Datetime"]).values

model=LinearRegression()
model.fit(train_df[cols].values,train_df["Consumption"].values.reshape(-1, 1))
preds=model.predict(valid_df[cols].values)

In [0]:
eval_df.append(eval_function("LR_2_7_w_dates",valid_df["Consumption"],preds))

RMSE:  2522.875757223404
RMSLE:  0.07974689639845284
MAE:  1665.06492012629




### LR Date categorical


In [0]:
cons_lagged=consumption_df.copy()
cons_lagged["Consumption_2_lag"]=cons_lagged["Consumption"].shift(24*2)
cons_lagged["Consumption_7_lag"]=cons_lagged["Consumption"].shift(24*7)
cons_lagged["Hour"]=cons_lagged["Hour"].apply(lambda x:x.split(":")[0]).astype(int)
cons_lagged["Day"]=cons_lagged["Date"].apply(lambda x:x.split(".")[0]).astype(int)
cons_lagged["Month"]=cons_lagged["Date"].apply(lambda x:x.split(".")[1]).astype(int)
cons_lagged["weekday"]=pd.to_datetime(cons_lagged["Date"]).apply(lambda x: x.weekday()).astype(int)
cons_lagged=cons_lagged.dropna()
cons_lagged

,Date,Hour,Consumption,Datetime,Consumption_2_lag,Consumption_7_lag,Day,Month,weekday
168,07.01.2016,0,28763.95,2016-07-01 00:00:00,29244.38,29590.88,7,1,4
169,07.01.2016,1,27284.84,2016-07-01 01:00:00,27637.23,27785.43,7,1,4
170,07.01.2016,2,26321.95,2016-07-01 02:00:00,26335.78,26516.53,7,1,4
171,07.01.2016,3,25748.49,2016-07-01 03:00:00,25870.23,26091.62,7,1,4
172,07.01.2016,4,25636.58,2016-07-01 04:00:00,25975.07,25871.54,7,1,4
...,...,...,...,...,...,...,...,...,...
36571,03.03.2020,19,37936.79,2020-03-03 19:00:00,35049.21,39283.70,3,3,1
36572,03.03.2020,20,37553.37,2020-03-03 20:00:00,34901.84,38791.40,3,3,1
36573,03.03.2020,21,36581.73,2020-03-03 21:00:00,34225.14,37588.86,3,3,1
36574,03.03.2020,22,35434.36,2020-03-03 22:00:00,33257.06,36290.64,3,3,1


In [0]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack

cat_features=["Hour","Day","Month","weekday"]
linear_enc=OneHotEncoder()
one_hot_encoded=linear_enc.fit_transform(cons_lagged[cat_features]).toarray()
normal_feats=cons_lagged.columns.difference(cat_features).tolist()
cons_lagged=np.hstack([cons_lagged[normal_feats],one_hot_encoded])
cons_lagged=pd.DataFrame(cons_lagged,columns=normal_feats+np.concatenate([[cat_features[i]+"_"+str(j) for j in cats_]for i,cats_ in enumerate(linear_enc.categories_)]).tolist())

In [0]:
train_df=cons_lagged.loc[cons_lagged["Datetime"]<=pd.to_datetime("2019-03-03 23:00:00")]
valid_df=cons_lagged.loc[cons_lagged["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]

In [0]:
from sklearn.linear_model import LinearRegression

cons_lagged=cons_lagged.dropna()

cols=train_df.columns.difference(["Date","Consumption","Datetime"]).values

model=LinearRegression()
model.fit(train_df[cols].values,train_df["Consumption"].values.reshape(-1, 1))
preds=model.predict(valid_df[cols].values)

In [0]:
eval_df.append(eval_function("LR_2_7_w_dates",valid_df["Consumption"],preds))

RMSE:  2411.7410279998844
RMSLE:  0.0762868233130247
MAE:  1592.3783633510911


### LR Date cyclical

In [0]:
cons_lagged=consumption_df.copy()
cons_lagged["Consumption_2_lag"]=cons_lagged["Consumption"].shift(24*2)
cons_lagged["Consumption_7_lag"]=cons_lagged["Consumption"].shift(24*7)
cons_lagged["Hour"]=cons_lagged["Hour"].apply(lambda x:x.split(":")[0]).astype(int)
cons_lagged["Day"]=cons_lagged["Date"].apply(lambda x:x.split(".")[0]).astype(int)
cons_lagged["Month"]=cons_lagged["Date"].apply(lambda x:x.split(".")[1]).astype(int)
cons_lagged["weekday"]=pd.to_datetime(cons_lagged["Date"]).apply(lambda x: x.weekday()).astype(int)
cons_lagged=cons_lagged.dropna()
cons_lagged

,Date,Hour,Consumption,Datetime,Consumption_2_lag,Consumption_7_lag,Day,Month,weekday
168,07.01.2016,0,28763.95,2016-07-01 00:00:00,29244.38,29590.88,7,1,4
169,07.01.2016,1,27284.84,2016-07-01 01:00:00,27637.23,27785.43,7,1,4
170,07.01.2016,2,26321.95,2016-07-01 02:00:00,26335.78,26516.53,7,1,4
171,07.01.2016,3,25748.49,2016-07-01 03:00:00,25870.23,26091.62,7,1,4
172,07.01.2016,4,25636.58,2016-07-01 04:00:00,25975.07,25871.54,7,1,4
...,...,...,...,...,...,...,...,...,...
36571,03.03.2020,19,37936.79,2020-03-03 19:00:00,35049.21,39283.70,3,3,1
36572,03.03.2020,20,37553.37,2020-03-03 20:00:00,34901.84,38791.40,3,3,1
36573,03.03.2020,21,36581.73,2020-03-03 21:00:00,34225.14,37588.86,3,3,1
36574,03.03.2020,22,35434.36,2020-03-03 22:00:00,33257.06,36290.64,3,3,1


In [0]:
cons_lagged['Hour_sin'] = np.sin(2 * np.pi * cons_lagged['Hour']/23.0)
cons_lagged['Hour_cos'] = np.cos(2 * np.pi * cons_lagged['Hour']/23.0)

cons_lagged['Day_sin'] = np.sin(2 * np.pi * cons_lagged['Day']/31.0)
cons_lagged['Day_cos'] = np.cos(2 * np.pi * cons_lagged['Day']/31.0)

cons_lagged['Month_sin'] = np.sin(2 * np.pi * cons_lagged['Month']/12.0)
cons_lagged['Month_cos'] = np.cos(2 * np.pi * cons_lagged['Month']/12.0)

cons_lagged['weekday_sin'] = np.sin(2 * np.pi * cons_lagged['weekday']/6.0)
cons_lagged['weekday_cos'] = np.cos(2 * np.pi * cons_lagged['weekday']/6.0)

del cons_lagged['Hour'],cons_lagged['Day'],cons_lagged['Month'],cons_lagged['weekday']

In [0]:
train_df=cons_lagged.loc[cons_lagged["Datetime"]<=pd.to_datetime("2019-03-03 23:00:00")]
valid_df=cons_lagged.loc[cons_lagged["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]

In [0]:
from sklearn.linear_model import LinearRegression

cons_lagged=cons_lagged.dropna()

cols=train_df.columns.difference(["Date","Consumption","Datetime"]).values

model=LinearRegression()
model.fit(train_df[cols].values,train_df["Consumption"].values.reshape(-1, 1))
preds=model.predict(valid_df[cols].values)

In [0]:
eval_df.append(eval_function("LR_2_7_w_dates",valid_df["Consumption"],preds))

RMSE:  2546.609637825625
RMSLE:  0.08020123457490627
MAE:  1689.0600541092615


In [0]:
cons_lagged

,Date,Hour,Consumption,Datetime,Consumption_2_lag,Consumption_7_lag,Day,Month,weekday,Hour_sin,Hour_cos,Day_sin,Day_cos,Month_sin,Month_cos,weekday_sin,weekday_cos
168,07.01.2016,0,28763.95,2016-07-01 00:00:00,29244.38,29590.88,7,1,4,0.000000e+00,1.000000,0.988468,0.151428,0.5,8.660254e-01,-0.866025,-0.5
169,07.01.2016,1,27284.84,2016-07-01 01:00:00,27637.23,27785.43,7,1,4,2.697968e-01,0.962917,0.988468,0.151428,0.5,8.660254e-01,-0.866025,-0.5
170,07.01.2016,2,26321.95,2016-07-01 02:00:00,26335.78,26516.53,7,1,4,5.195840e-01,0.854419,0.988468,0.151428,0.5,8.660254e-01,-0.866025,-0.5
171,07.01.2016,3,25748.49,2016-07-01 03:00:00,25870.23,26091.62,7,1,4,7.308360e-01,0.682553,0.988468,0.151428,0.5,8.660254e-01,-0.866025,-0.5
172,07.01.2016,4,25636.58,2016-07-01 04:00:00,25975.07,25871.54,7,1,4,8.878852e-01,0.460065,0.988468,0.151428,0.5,8.660254e-01,-0.866025,-0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36571,03.03.2020,19,37936.79,2020-03-03 19:00:00,35049.21,39283.70,3,3,1,-8.878852e-01,0.460065,0.571268,0.820763,1.0,6.123234e-17,0.866025,0.5
36572,03.03.2020,20,37553.37,2020-03-03 20:00:00,34901.84,38791.40,3,3,1,-7.308360e-01,0.682553,0.571268,0.820763,1.0,6.123234e-17,0.866025,0.5
36573,03.03.2020,21,36581.73,2020-03-03 21:00:00,34225.14,37588.86,3,3,1,-5.195840e-01,0.854419,0.571268,0.820763,1.0,6.123234e-17,0.866025,0.5
36574,03.03.2020,22,35434.36,2020-03-03 22:00:00,33257.06,36290.64,3,3,1,-2.697968e-01,0.962917,0.571268,0.820763,1.0,6.123234e-17,0.866025,0.5


In [0]:
residuals=model.predict(train_df[cols].values,train_df["Consumption"].)
